Here we attempt to use a python library called textblob which is mainly used for the transformation of text data. As we have identified, tweets commonly have many spelling and grammar mistakes that needs to be rectified.

In [6]:
from textblob import TextBlob
import pandas as pd
import numpy as np
import re
import pandas as pd
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.model_selection import train_test_split
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import spacy
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package punkt to C:\Users\Josiah
[nltk_data]     Lee\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Josiah
[nltk_data]     Lee\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
# Load the dataset
data = pd.read_csv('Twitter Sentiment/dataset.csv', header=None)
data.columns = ['labels', 'text']
print(len(data))
print(data.head())

100000
   labels                                               text
0       0  @switchfoot http://twitpic.com/2y1zl - Awww, t...
1       0  is upset that he can't update his Facebook by ...
2       0  @Kenichan I dived many times for the ball. Man...
3       0    my whole body feels itchy and like its on fire 
4       0  @nationwideclass no, it's not behaving at all....


TextBlob actually hs its own sentiment analysis method. So I tried my luck with it. The output of the polarity ranges from -1 to 1 and the huge number of 0s indicate that there is actually a lot of data that the model is unable to classify. Tired. 

In [8]:
predictions = []

for i in range(24999, 75000):
    string = data.text[i]
    tb = TextBlob(string)
    prob = tb.sentiment.polarity
    predictions.append(prob)

In [9]:
len(list(filter(lambda x: x==0, predictions)))

17789

What I intend to do would be similar to what we previously had but instead I'll just add an autocorrect and see what happens.

In [10]:
data = pd.read_csv('Twitter Sentiment/spelt.csv', header=0, index_col=0)
data

,labels,text,cleaned_text,spelled
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",awww that's a bummer you shoulda got david ca...,www that's a summer you should got david carr...
1,0,is upset that he can't update his Facebook by ...,is upset that he can't update his facebook by ...,is upset that he can't update his facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...,i dived many times for the ball managed to sa...,i dived many times for the ball managed to sa...
3,0,my whole body feels itchy and like its on fire,my whole body feels itchy and like its on fire,my whole body feels itch and like its on fire
4,0,"@nationwideclass no, it's not behaving at all....",no it's not behaving at all i'm mad why am i ...,no it's not behaving at all i'm mad why am i ...
...,...,...,...,...
99995,1,Now need 8 followers to compleate 1000 Follow...,now need followers to compleate follow,now need followers to complete follow
99996,1,I knew I had to explain something to my friend...,i knew i had to explain something to my friend...,i knew i had to explain something to my friend...
99997,1,done tweeting..... til tomorrow..,done tweeting til tomorrow,done meeting til tomorrow
99998,1,@cmozilo Act II set is pretty breath-taking -L...,act ii set is pretty breath taking love the r...,act ii set is pretty breath taking love the r...


In [11]:
def clean_text(text):
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'@[A-Za-z0-9_]+', '', text)  # Remove mentions
    text = re.sub(r'#', '', text)  # Remove hashtags
    text = re.sub(r'[^A-Za-z\']+', ' ', text)  # Keep only letters
    text = text.lower()  # Convert to lowercase
    return text

data['cleaned_text'] = data['text'].apply(clean_text)

Using autocorrect library as it runs the fastest and most autocorrect libraries would perform equally 

In [12]:
from autocorrect import Speller

check = Speller(lang = 'en')

In [13]:
sample_txt = []

for sentence in data.cleaned_text:
    sample_txt.append(check(sentence))

data['spelled'] = sample_txt

KeyboardInterrupt: 

saving the data

In [ ]:
data.to_csv('spelt.csv')

removing of apostrophe

In [ ]:
def clean_text(text):
    text = re.sub(r'\'', '', text)
    return text

data['apostrophe'] = data['spelled'].apply(clean_text)
data.apostrophe

0         www thats a summer you should got david carr ...
1        is upset that he cant update his facebook by t...
2         i dived many times for the ball managed to sa...
3           my whole body feels itch and like its on fire 
4         no its not behaving at all im mad why am i he...
                               ...                        
99995               now need followers to complete follow 
99996    i knew i had to explain something to my friend...
99997                           done meeting til tomorrow 
99998     act ii set is pretty breath taking love the r...
99999    if you dont have an attire account to sell you...
Name: apostrophe, Length: 100000, dtype: object

In [ ]:
data['tokens'] = data['apostrophe'].apply(word_tokenize)
data.tokens

0        [www, thats, a, summer, you, should, got, davi...
1        [is, upset, that, he, cant, update, his, faceb...
2        [i, dived, many, times, for, the, ball, manage...
3        [my, whole, body, feels, itch, and, like, its,...
4        [no, its, not, behaving, at, all, im, mad, why...
                               ...                        
99995         [now, need, followers, to, complete, follow]
99996    [i, knew, i, had, to, explain, something, to, ...
99997                       [done, meeting, til, tomorrow]
99998    [act, ii, set, is, pretty, breath, taking, lov...
99999    [if, you, dont, have, an, attire, account, to,...
Name: tokens, Length: 100000, dtype: object

In [ ]:
nlp = spacy.load("en_core_web_sm")

def lemmatize_words(word_list):
    doc = nlp(" ".join(word_list))  # Join the stemmed tokens into a string
    lemmatized_text = [token.lemma_ for token in doc]
    return lemmatized_text

data['lemmatized_token'] = data['tokens'].apply(lemmatize_words)
data.lemmatized_token

0        [www, that, s, a, summer, you, should, got, da...
1        [be, upset, that, he, ca, nt, update, his, fac...
2        [I, dive, many, time, for, the, ball, manage, ...
3        [my, whole, body, feel, itch, and, like, its, ...
4        [no, its, not, behave, at, all, I, m, mad, why...
                               ...                        
99995          [now, need, follower, to, complete, follow]
99996    [I, know, I, have, to, explain, something, to,...
99997                         [do, meeting, til, tomorrow]
99998    [act, ii, set, be, pretty, breath, take, love,...
99999    [if, you, do, nt, have, an, attire, account, t...
Name: lemmatized_token, Length: 100000, dtype: object

In [ ]:
stop_words = set(stopwords.words('english'))
data['filtered_tokens'] = data['lemmatized_token'].apply(lambda tokens: [word for word in tokens if word not in stop_words])
data['filtered_tokens'] = data['filtered_tokens'].apply(lambda tokens: [word.lower() for word in tokens])


data['filtered_tokens']

0              [www, summer, got, david, carr, third, day]
1        [upset, ca, nt, update, facebook, texte, might...
2        [i, dive, many, time, ball, manage, save, rest...
3                    [whole, body, feel, itch, like, fire]
4                      [behave, i, mad, i, i, ca, nt, see]
                               ...                        
99995                   [need, follower, complete, follow]
99996    [i, know, i, explain, something, friend, say, ...
99997                             [meeting, til, tomorrow]
99998    [act, ii, set, pretty, breath, take, love, rea...
99999    [nt, attire, account, sell, fun, thing, i, sug...
Name: filtered_tokens, Length: 100000, dtype: object

In [ ]:
# Split the dataset

X_train, X_test, y_train, y_test = train_test_split(data.filtered_tokens, data.labels, stratify = data.labels, test_size=0.2, random_state=42)

In [ ]:
vectorizer = CountVectorizer(binary=True, ngram_range=(1, 2), stop_words='english')
sentences = [' '.join(words) for words in X_train]

# Fit and transform the sentences
one_hot_encoded = vectorizer.fit_transform(sentences)

       shastri  invoke  mnras  priority  minute  stole  discount  airbender  \
0            0       0      0         0       0      0         0          0   
1            0       0      0         0       0      0         0          0   
2            0       0      0         0       0      0         0          0   
3            0       0      0         0       0      0         0          0   
4            0       0      0         0       0      0         0          0   
...        ...     ...    ...       ...     ...    ...       ...        ...   
99995        0       0      0         0       0      0         0          0   
99996        0       0      0         0       0      0         0          0   
99997        0       0      0         0       0      0         0          0   
99998        0       0      0         0       0      0         0          0   
99999        0       0      0         0       0      0         0          0   

       gooooooood  nuit  ...  davids  crankiness  i

In [ ]:
import sklearn
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest

test = SelectKBest(score_func=chi2, k=3000)
X_train_new=test.fit_transform(one_hot_encoded, y_train)

In [ ]:
clf = sklearn.linear_model.LogisticRegressionCV(random_state = 42, cv = 10).fit(X_train_new, y_train)


c:\Users\Josiah Lee\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\Josiah Lee\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-le

In [ ]:
sen = [' '.join(words) for words in X_test]
X_test_new = vectorizer.transform(sen)
X_test_new = test.transform(X_test_new)
y_pred = clf.predict(X_test_new)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Classification Report:\n{classification_rep}')

Accuracy: 0.7572
Classification Report:
              precision    recall  f1-score   support

           0       0.77      0.74      0.75     10000
           1       0.75      0.78      0.76     10000

    accuracy                           0.76     20000
   macro avg       0.76      0.76      0.76     20000
weighted avg       0.76      0.76      0.76     20000

